In [1]:
from datasets import load_dataset
import polars as pl
import os
import cynde.functional as cf

In [2]:
# Get the directory above the current directory
above_dir = os.path.dirname(os.getcwd())

# Specify the cache directory as 'data' subdirectory within the current directory
cache_dir = os.path.join(above_dir, "cache")
print(cache_dir)

c:\Users\Tommaso\Documents\Dev\Cynde\cache


In [3]:
dataset_name = "OpenHermes-2.5"
dataset_path = os.path.join(cache_dir, dataset_name)


In [4]:
df = pl.read_parquet(dataset_path+"_filtered.parquet")
df.head(1)


conversations,source
list[struct[3]],str
"[{""human"",""Given the following list of words, categorize them into 5 categories by similarity: ['Heart', 'Liver', 'Kidney', 'Brain', 'Stomach', 'Skin', 'Lung', 'Muscle', 'Intestine', 'Bone', 'Pancreas', 'Blood', 'Eye', 'Ear', 'Nose', 'Mouth', 'Esophagus', 'Throat', 'Spinal Cord', 'Bladder']"",null}, {""gpt"",""Here are the given words categorized into 5 groups based on their similarity: 1. Vital Organs: - Heart - Liver - Kidney - Lung - Pancreas 2. Digestive System Components: - Stomach - Intestine - Esophagus - Throat - Mouth 3. Sensory Organs: - Eye - Ear - Nose 4. Support and Movement: - Bone - Muscle - Spinal Cord - Skin 5. Body Fluids and Waste Management: - Blood - Bladder"",null}, … {""gpt"",""1) Vital Organs: - Diseases or disorders affecting one vital organ can impact the functioning of others. For example, heart failure can lead to fluid buildup in the lungs, affecting their function. Liver disease can cause toxins to accumulate in the bloodstream, potentially affecting brain function. Kidney disease can lead to electrolyte imbalances, which can impact heart function. - Organ transplantation involves replacing a damaged organ with a healthy one from a donor. Examples include heart, liver, kidney, lung, and pancreas transplants. Challenges include finding compatible donors, managing immune system rejection, and addressing surgical complications. 2) Digestive System Components: - Common digestive disorders include gastroesophageal reflux disease (GERD), irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), and peptic ulcers. Causes may include genetic factors, lifestyle choices, stress, infections, or immune system dysfunction. - The gut microbiome comprises trillions of microorganisms that aid in digestion, nutrient absorption, and immune system function. It can also influence mental health and contribute to the prevention or development of various diseases. 3) Sensory Organs: - Optical illusions work by exploiting the brain's visual processing system, causing it to misinterpret sensory information. They reveal that our perception of reality can be influenced by factors such as context, contrast, and patterns. - Animals' sensory organs can differ significantly from humans', offering unique capabilities. For example, dogs have a heightened sense of smell, eagles have exceptional long-distance vision, and bats use echolocation for navigation and foraging. 4) Support and Movement: - Exercise and physical activity promote bone and muscle health by stimulating bone growth, increasing bone density, and strengthening muscles. This helps prevent age-related bone loss and reduces the risk of injury. - Spinal cord injuries can have varying effects depending on the location and severity of the injury. Injuries can result in temporary or permanent loss of sensation, mobility, and other bodily functions. Paraplegia (paralysis of the legs and lower body) and quadriplegia (paralysis of all four limbs) are examples of potential outcomes of spinal cord injuries. 5) Body Fluids and Waste Management: - Blood types affect transfusions and organ transplants because of the presence of specific antigens on red blood cells. Incompatible blood types can cause an immune response, resulting in transfusion reactions or organ rejection. - Common bladder disorders include urinary incontinence, urinary tract infections (UTIs), interstitial cystitis, and bladder cancer. Treatments and management strategies can vary, including medication, lifestyle changes, physical therapy, or surgical interventions."",null}]","""caseus_custom"""


In [5]:
df["source"].value_counts()

source,count
str,u32
"""caseus_custom""",2688
"""LMSys Chatbot …",3136


In [6]:
df = df.with_columns(pl.col("conversations").list.eval(pl.element().struct.json_encode()).list.join("\n"),(pl.col("source")=="caseus_custom").alias("target"))

In [7]:
#check if the embedded df already exist at dataset_path+"_embedded.parquet"
if not os.path.exists(dataset_path+"_embedded_small_large.parquet"):
    embedded_df = cf.embed_columns(df, ["conversations"],models=["text-embedding-3-small","text-embedding-3-large"])
    embedded_df.write_parquet(dataset_path+"_embedded_small_large.parquet")
else:
    embedded_df = pl.read_parquet(dataset_path+"_embedded_small_large.parquet")

In [8]:
df["target"].value_counts()

target,count
bool,u32
false,3136
true,2688


In [9]:
cf.vanilla_kfold(embedded_df,group=None,k=5)

cv_index,fold_0,fold_1,fold_2,fold_3,fold_4
u32,str,str,str,str,str
1260,"""test""","""train""","""train""","""train""","""train"""
2223,"""test""","""train""","""train""","""train""","""train"""
1531,"""test""","""train""","""train""","""train""","""train"""
5245,"""test""","""train""","""train""","""train""","""train"""
3553,"""test""","""train""","""train""","""train""","""train"""
3894,"""test""","""train""","""train""","""train""","""train"""
1319,"""test""","""train""","""train""","""train""","""train"""
2481,"""test""","""train""","""train""","""train""","""train"""
3074,"""test""","""train""","""train""","""train""","""train"""


df:pl.DataFrame, cv_type: Tuple[str,str], group_outer:List[str],k_outer:int,group_inner:List[str],k_inner:int,r_outer:int =1, r_inner:int =1):

In [10]:
models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5},{"n_estimators": 50, "max_depth": 10}]}
inputs =[{"numerical":["conversations_text-embedding-3-small_embeddings"]},
         {"numerical":["conversations_text-embedding-3-large_embeddings"]},
         {"numerical":["conversations_text-embedding-3-small_embeddings","conversations_text-embedding-3-large_embeddings"]}]

In [11]:
results,pred=cf.train_nested_cv(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=1,
                     r_inner=1,
                     save_name="test")


cv_type: stratified
cv_type: stratified
['cv_index', 'target', 'outer_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_1', 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1', 'outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_1']
Accuracy Test: 0.8602335164835165
Accuracy Val: 0.864010989010989
MCC Test: 0.7187789269010901
MCC Val: 0.726297790261728
Total CLS time: 00:00:00
Accuracy Test: 0.9220467032967034
Accuracy Val: 0.926510989010989
MCC Test: 0.8432911376595675
MCC Val: 0.8521072730197063
Total CLS time: 00:00:01
Accuracy Test: 0.8756868131868132
Accuracy Val: 0.882554945054945
MCC Test: 0.7500827692637394
MCC Val: 0.763729795085458
Total CLS time: 00:00:00
Accuracy Test: 0.9416208791208791
Accuracy Val: 0.9340659340659341


In [16]:
summary = cf.results_summary(results,by_test_fold=True)

In [17]:
for res in summary.rows(named=True):
    print(res["input_features_name"],res["classifier_hp"],res["mcc_val"])

conversations_text-embedding-3-large_embeddings n_estimators_50_max_depth_10 0.8781529260557548
conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings n_estimators_50_max_depth_10 0.8768594671301411
conversations_text-embedding-3-small_embeddings n_estimators_50_max_depth_10 0.8341343669608386
conversations_text-embedding-3-large_embeddings n_estimators_10_max_depth_5 0.7697169185685329
conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings n_estimators_10_max_depth_5 0.7624967568562282
conversations_text-embedding-3-small_embeddings n_estimators_10_max_depth_5 0.7215598990854284
